In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
import warnings

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)

In [3]:
def plot_columns(df):
    colors = plt.cm.tab20(range(df.shape[1]))  

    for i, column in enumerate(df.columns):
        color = colors[i % 24]

        plt.figure()
        plt.plot(df.index, df[column], color=color)
        plt.xlabel('Ano')
        plt.ylabel('Valor')
        plt.title(column)
        plt.xticks(rotation=45)

    plt.show()
    
    
def plot_linear_regression(column, linear_a, linear_b, linear_a_half, linear_b_half):
    plt.figure(figsize=(10, 6))
    plt.scatter(column.index, column, label='Original Values')
    plt.plot(column.index, linear_a * column.index + linear_b, color='red', label='Linear Regression')
    plt.plot(column.index, linear_a_half * column.index + linear_b_half, color='green', label='Recent Linear Regression')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.title('Original Values vs. Linear Regression vs. Recent Linear Regression')
    plt.grid(True)
    plt.show()

In [4]:
def centered_moving_average(column, window_action, window_size):
    start_index, end_index = window_action
    data = column.to_numpy()

    data_length = len(data)

    for i in range(start_index, end_index + 1):
        if not np.isnan(data[i]):
            init = data[i]
            start = max(0, i - window_size)
            end = min(data_length, i + window_size + 1)
            window = data[start:end]
            valid_values = [value for value in window if not pd.isna(value)]
            if len(valid_values) > 0:
                data[i] = np.mean(valid_values)

    modified_column = pd.Series(data, name=column.name)

    return modified_column

In [5]:
# Polynomial Interpolation data inputation method
def four_ends_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        for i in range(4):
            column = first_filler(column, i, len(column) - 1 - i)
            column = last_filler(column, i, len(column) - 1 - i)
    return column

def four_ends_linear_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        size = len(column) - 1
        column = first_filler(column, 0, size)
        column = last_filler(column, 0, size)
        for i in range(1, 4):
            value = column.at[i]
            if (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_first = i - 1
                end_first = next_valid_index(column, i)
                linear_a = (column.at[end_first] - column.at[i - 1])/(end_first - start_first)
                linear_b = column.at[start_first] - linear_a*start_first
                column.at[i] = linear_a*i + linear_b
            x = size - 4 + i
            value = column.at[x]
            if (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_last = prev_valid_index(column, x)
                end_last = next_valid_index(column, x)
                gap = list(range(start_last + 1, end_last))
                linear_a = (column.at[end_last] - column.at[start_last])/(end_last - start_last)
                linear_b = column.at[start_last] - linear_a*start_last
                column.at[x] = linear_a*x + linear_b
    return column

def first_filler(column, init, end):
    if pd.isnull(column.at[init]) or np.isnan(column.at[init]):
        first_valid_index = 0
        while first_valid_index <= end and (pd.isnull(column.at[first_valid_index]) or 
                                             np.isnan(column.at[first_valid_index])):
            first_valid_index += 1
        if first_valid_index > end:
            column.at[init] = 0.0
        else:
            column.at[init] = column.at[first_valid_index]
    return column

def extract_valid_values(column):
    valid_values = []
    valid_indexes = []
    for index, value in enumerate(column):
        if not pd.isnull(value) and not np.isnan(value):
            valid_values.append(value)
            valid_indexes.append(index)
    extracted_column = pd.Series(
        valid_values,
        index=valid_indexes
    )
    return extracted_column

def linear_regression_first_four_filler(column):
    size = len(column) - 1
    processed_column = extract_valid_values(column)
    mean_valid_values_quarter = processed_column.mean()/4
    mean_valid_values_quarter_upper = processed_column.mean() + mean_valid_values_quarter
    
    model = sm.OLS(processed_column, sm.add_constant(processed_column.index))
    result = model.fit()
    linear_a = result.params[1]
    linear_b = result.params['const']
    half_column = processed_column[:(len(processed_column) // 2)]
    model2 = sm.OLS(half_column, sm.add_constant(half_column.index))
    result2 = model2.fit()
    linear_a_half = result2.params[1]
    linear_b_half = result2.params['const']
    
    for i in range(0,4):
        if pd.isnull(column.at[i]) or np.isnan(column.at[i]):
            projected = ((linear_a*i + linear_b) + (linear_a_half*i + linear_b_half))/2
            if projected < mean_valid_values_quarter:
                column.at[i] = mean_valid_values_quarter
            elif projected > mean_valid_values_quarter_upper:
                column.at[i] = mean_valid_values_quarter_upper
            else:
                column.at[i] = projected
        x = size - 4 + i
        if pd.isnull(column.at[x]) or np.isnan(column.at[x]):
            projected = ((linear_a*i + linear_b) + (linear_a_half*i + linear_b_half))/2
            if projected < mean_valid_values_quarter:
                column.at[x] = mean_valid_values_quarter
            elif projected > mean_valid_values_quarter_upper:
                column.at[x] = mean_valid_values_quarter_upper
            else:
                column.at[x] = projected
    plot_linear_regression(column, linear_a, linear_b, linear_a_half, linear_b_half)
    return column
    
def last_filler(column, init, end):
    if pd.isnull(column.at[end]) or np.isnan(column.at[end]):
        last_valid_index = len(column) -1
        while last_valid_index >= init and (pd.isnull(column.at[last_valid_index]) or 
                                         np.isnan(column.at[last_valid_index])):
            last_valid_index -= 1
        if last_valid_index < init:
            column.at[end] = 0.0
        else:
            column.at[end] = column.at[last_valid_index]
    return column
  
def neareast_four_indexes(column, start, end):
    return prev_four_valid_indexes(column, start) + next_four_valid_indexes(column, end)

def prev_four_valid_indexes(column, pos):
    prev_index = pos
    index_prev_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_prev_list[3 - i] = prev_index
            prev_index -= 1
            while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
                prev_index -= 1
    return index_prev_list  

def next_four_valid_indexes(column, pos):
    next_index = pos
    index_next_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_next_list[i] = next_index
            next_index += 1
            while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
                next_index += 1
    return index_next_list  

def prev_valid_index(column, pos):
    prev_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        prev_index -= 1
        while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
            prev_index -= 1
    return prev_index  

def next_valid_index(column, pos):
    next_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        next_index += 1
        while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
            next_index += 1
    return next_index

def interpolating_polynomial(nearest_indexes, nearest_values):
    coeffs = np.polyfit(nearest_indexes, nearest_values, 3)
    return np.poly1d(coeffs)

def polynomial_filler(column):
    size = len(column) -1
    if column.isnull().any() and column.isna().any():
        for i, value in enumerate(column):
            if (i > 3 and i < size - 3) and (pd.isnull(value) or np.isnan(value) or (value == '')):
                nearest_values = []
                start = prev_valid_index(column, i)
                end = next_valid_index(column, i)
                gap = list(range(start + 1, end))
                nearest_indexes = neareast_four_indexes(column, start, end)
                for index in nearest_indexes:
                    nearest_values.append(column.at[index])
                polynomial = interpolating_polynomial(nearest_indexes, nearest_values)
                for x in gap:
                    column.at[x] = polynomial(x)
                plot_polynomial(polynomial, nearest_indexes[0], nearest_indexes[len(nearest_indexes) - 1], 50, gap, nearest_indexes, column.name)
    return column

def plot_polynomial(polynomial, start, end, precision, interpolated_list, nearest_indexes, column_name):
    x = np.linspace(start, end, precision)
    y = polynomial(x)

    plt.figure()

    plt.plot(x, y)

    interpolated_x = interpolated_list  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='red')
    interpolated_x = nearest_indexes  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='green')
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(column_name)

    plt.show()
    
def replace_negative_with_float_zeros(x):
    return 0.0 if x < 0 else x

In [6]:
def read_csv_replace_missing_by_polynomial(file_path, skiped_columns = 0, skiped_rows = 0):
    with open(file_path, 'r') as f:
        
        df = pd.read_csv(file_path, skiprows=skiped_rows)

        i = 0
        for col in df.columns: # Elimina missing values por médias e transforma em float
            if i > skiped_columns:
                df[col] = df[col].replace('-9999', np.nan)
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].replace('', np.nan).astype(float)
                df[col] = four_ends_filler(df[col])
                df[col] = centered_moving_average(df[col], (0, len(df[col]) - 1), 24) #2 anos
                df[col] = polynomial_filler(df[col]) 
            i += 1
        return df

In [7]:
df = pd.read_csv('deposito_poupanca.csv')
df.set_index('Estado', inplace=True)
df = df.drop(['UF','Código'], axis=1)
df = df.T
df.rename(columns={'Estado': 'UF'}, inplace=True)
df

Estado,Acre,Alagoas,Amazonas,Amapá,Bahia,Ceará,Distrito Federal,Espírito Santo,Goiás,Maranhão,...,Paraná,Rio de Janeiro,Rio Grande do Norte,Rondônia,Roraima,Rio Grande do Sul,Santa Catarina,Sergipe,São Paulo,Tocantins
2003,247158.775150,1.594356e+06,1.284537e+06,205197.602881,7.782609e+06,4.636226e+06,4.896482e+06,4.069229e+06,3.935260e+06,1.609842e+06,...,1.312029e+07,3.675288e+07,1.843384e+06,5.714165e+05,137154.814617,2.046281e+07,8.001967e+06,1.618045e+06,8.652299e+07,3.810181e+05
2004,259422.782134,1.638129e+06,1.370642e+06,208807.697157,8.316430e+06,4.888193e+06,5.348840e+06,4.370580e+06,4.260107e+06,1.723195e+06,...,1.331100e+07,3.667942e+07,1.989660e+06,6.264824e+05,147387.904447,2.113896e+07,8.436550e+06,1.670497e+06,8.734721e+07,4.058694e+05
2005,291267.223989,1.644989e+06,1.398651e+06,214497.406590,8.416032e+06,5.100767e+06,5.398898e+06,4.452244e+06,4.147822e+06,1.959751e+06,...,1.283067e+07,3.543637e+07,2.042178e+06,6.449006e+05,152846.938566,2.090356e+07,8.375574e+06,1.714598e+06,8.530959e+07,4.375903e+05
2006,320070.059633,1.739302e+06,1.537541e+06,233930.355555,9.032092e+06,4.925766e+06,5.611119e+06,4.863575e+06,4.531156e+06,2.237789e+06,...,1.317850e+07,3.587359e+07,2.252854e+06,7.088486e+05,221087.202784,2.143924e+07,8.566029e+06,1.811475e+06,8.725309e+07,5.729654e+05
2007,359397.120658,1.985817e+06,1.819514e+06,294780.372830,1.066705e+07,5.735004e+06,6.904643e+06,5.676563e+06,5.752881e+06,2.648197e+06,...,1.601391e+07,4.077419e+07,2.573341e+06,9.005881e+05,275519.839741,2.541255e+07,1.022738e+07,2.140601e+06,1.017343e+08,6.550792e+05
2008,475998.129219,2.108899e+06,2.087573e+06,411236.260883,1.170429e+07,6.172585e+06,7.218711e+06,5.445493e+06,6.236123e+06,3.044493e+06,...,1.696356e+07,4.245869e+07,2.640785e+06,1.044762e+06,471886.078343,2.636300e+07,1.385752e+07,2.345188e+06,1.082609e+08,6.849772e+05
2009,574349.795459,2.341276e+06,2.230064e+06,389457.256906,1.316888e+07,6.827420e+06,7.908179e+06,5.850375e+06,6.792006e+06,3.544671e+06,...,1.864090e+07,4.779394e+07,2.776817e+06,1.205506e+06,414799.404824,2.860932e+07,1.428115e+07,2.522125e+06,1.199192e+08,7.441381e+05
2010,599379.011000,2.950453e+06,2.538483e+06,419624.175000,1.505146e+07,7.756349e+06,8.741042e+06,6.496175e+06,7.783633e+06,3.633747e+06,...,2.083445e+07,5.191714e+07,3.021951e+06,1.470233e+06,401300.112000,3.019919e+07,1.489502e+07,2.892812e+06,1.298677e+08,9.749694e+05
2011,577513.618102,3.093400e+06,2.618416e+06,423914.006445,1.564025e+07,8.106733e+06,8.741836e+06,7.045605e+06,8.235940e+06,3.797295e+06,...,2.214097e+07,5.293384e+07,3.102611e+06,1.559391e+06,410851.764119,3.021739e+07,1.572651e+07,2.929514e+06,1.310444e+08,1.038873e+06
2012,618593.348202,3.324983e+06,2.847033e+06,472371.694694,1.720493e+07,8.768239e+06,9.687777e+06,7.802161e+06,9.239086e+06,4.129860e+06,...,2.493568e+07,5.769342e+07,3.338127e+06,1.634068e+06,444921.920078,3.313447e+07,1.704920e+07,3.181092e+06,1.427618e+08,1.178756e+06


In [8]:
def dataframe_expanded_interpolation(df):
    expanded_df = pd.DataFrame(columns=df.columns)
    for i, (_, row) in enumerate(df.iterrows()):
        if i > 0:
            expanded_df = pd.concat([expanded_df, pd.DataFrame(columns=df.columns, index=[None] * 11)])
        expanded_df = pd.concat([expanded_df, pd.DataFrame(row).T])
    expanded_df = expanded_df.reset_index(drop=True)
    expanded_df.interpolate(method='linear', axis=0, inplace=True)

    return expanded_df

def add_ano_column(df):
    new_df = df.copy()
    ano_start = 2002
    for i in range(len(new_df)):
        if i % 12 == 0:
            ano_start += 1
        new_df.at[i, 'Ano'] =  str(ano_start)

    return new_df

def add_month(df):
    new_index = [] 
    for i, index_value in enumerate(df.index):
        iteration = i % 12 + 1
        new_index.append(f'{index_value}-{iteration}')
    df.index = new_index

    return df

In [9]:
pd.set_option('display.max_rows', None)
df_mo_2003 = dataframe_expanded_interpolation(df)
df_mo_2003 = add_ano_column(df_mo_2003)
df_mo_2003 = df_mo_2003.set_index('Ano', inplace=False)
df_mo_2003 = add_month(df_mo_2003)
df_mo_2003

Estado,Acre,Alagoas,Amazonas,Amapá,Bahia,Ceará,Distrito Federal,Espírito Santo,Goiás,Maranhão,...,Paraná,Rio de Janeiro,Rio Grande do Norte,Rondônia,Roraima,Rio Grande do Sul,Santa Catarina,Sergipe,São Paulo,Tocantins
2003-1,247158.775150,1.594356e+06,1.284537e+06,205197.602881,7.782609e+06,4.636226e+06,4.896482e+06,4.069229e+06,3.935260e+06,1.609842e+06,...,1.312029e+07,3.675288e+07,1.843384e+06,5.714165e+05,137154.814617,2.046281e+07,8.001967e+06,1.618045e+06,8.652299e+07,3.810181e+05
2003-2,248180.775732,1.598004e+06,1.291713e+06,205498.444071,7.827094e+06,4.657223e+06,4.934178e+06,4.094341e+06,3.962330e+06,1.619288e+06,...,1.313619e+07,3.674675e+07,1.855573e+06,5.760053e+05,138007.572103,2.051916e+07,8.038182e+06,1.622416e+06,8.659168e+07,3.830890e+05
2003-3,249202.776314,1.601652e+06,1.298888e+06,205799.285261,7.871579e+06,4.678221e+06,4.971875e+06,4.119454e+06,3.989401e+06,1.628734e+06,...,1.315208e+07,3.674063e+07,1.867763e+06,5.805941e+05,138860.329589,2.057551e+07,8.074397e+06,1.626787e+06,8.666036e+07,3.851600e+05
2003-4,250224.776896,1.605299e+06,1.306063e+06,206100.126450,7.916064e+06,4.699218e+06,5.009571e+06,4.144566e+06,4.016472e+06,1.638180e+06,...,1.316797e+07,3.673451e+07,1.879953e+06,5.851830e+05,139713.087075,2.063185e+07,8.110613e+06,1.631158e+06,8.672905e+07,3.872309e+05
2003-5,251246.777478,1.608947e+06,1.313239e+06,206400.967640,7.960549e+06,4.720215e+06,5.047268e+06,4.169679e+06,4.043542e+06,1.647626e+06,...,1.318386e+07,3.672839e+07,1.892142e+06,5.897718e+05,140565.844561,2.068820e+07,8.146828e+06,1.635529e+06,8.679773e+07,3.893019e+05
2003-6,252268.778060,1.612595e+06,1.320414e+06,206701.808830,8.005034e+06,4.741212e+06,5.084964e+06,4.194792e+06,4.070613e+06,1.657072e+06,...,1.319975e+07,3.672227e+07,1.904332e+06,5.943606e+05,141418.602046,2.074454e+07,8.183043e+06,1.639900e+06,8.686642e+07,3.913728e+05
2003-7,253290.778642,1.616242e+06,1.327590e+06,207002.650019,8.049519e+06,4.762209e+06,5.122661e+06,4.219904e+06,4.097683e+06,1.666518e+06,...,1.321565e+07,3.671615e+07,1.916522e+06,5.989494e+05,142271.359532,2.080089e+07,8.219259e+06,1.644271e+06,8.693510e+07,3.934438e+05
2003-8,254312.779224,1.619890e+06,1.334765e+06,207303.491209,8.094004e+06,4.783207e+06,5.160357e+06,4.245017e+06,4.124754e+06,1.675964e+06,...,1.323154e+07,3.671003e+07,1.928711e+06,6.035383e+05,143124.117018,2.085723e+07,8.255474e+06,1.648642e+06,8.700379e+07,3.955147e+05
2003-9,255334.779806,1.623538e+06,1.341940e+06,207604.332399,8.138489e+06,4.804204e+06,5.198054e+06,4.270130e+06,4.151824e+06,1.685410e+06,...,1.324743e+07,3.670391e+07,1.940901e+06,6.081271e+05,143976.874504,2.091358e+07,8.291689e+06,1.653013e+06,8.707247e+07,3.975856e+05
2003-10,256356.780388,1.627186e+06,1.349116e+06,207905.173588,8.182974e+06,4.825201e+06,5.235750e+06,4.295242e+06,4.178895e+06,1.694856e+06,...,1.326332e+07,3.669778e+07,1.953091e+06,6.127159e+05,144829.631990,2.096992e+07,8.327904e+06,1.657384e+06,8.714116e+07,3.996566e+05


In [10]:
def add_suffix_to_columns(df, suffix):
    df.columns = [col + suffix for col in df.columns]

add_suffix_to_columns(df_mo_2003, " - Depósito Poupança (R$)")
df_mo_2003

,Acre - Depósito Poupança (R$),Alagoas - Depósito Poupança (R$),Amazonas - Depósito Poupança (R$),Amapá - Depósito Poupança (R$),Bahia - Depósito Poupança (R$),Ceará - Depósito Poupança (R$),Distrito Federal - Depósito Poupança (R$),Espírito Santo - Depósito Poupança (R$),Goiás - Depósito Poupança (R$),Maranhão - Depósito Poupança (R$),...,Paraná - Depósito Poupança (R$),Rio de Janeiro - Depósito Poupança (R$),Rio Grande do Norte - Depósito Poupança (R$),Rondônia - Depósito Poupança (R$),Roraima - Depósito Poupança (R$),Rio Grande do Sul - Depósito Poupança (R$),Santa Catarina - Depósito Poupança (R$),Sergipe - Depósito Poupança (R$),São Paulo - Depósito Poupança (R$),Tocantins - Depósito Poupança (R$)
2003-1,247158.775150,1.594356e+06,1.284537e+06,205197.602881,7.782609e+06,4.636226e+06,4.896482e+06,4.069229e+06,3.935260e+06,1.609842e+06,...,1.312029e+07,3.675288e+07,1.843384e+06,5.714165e+05,137154.814617,2.046281e+07,8.001967e+06,1.618045e+06,8.652299e+07,3.810181e+05
2003-2,248180.775732,1.598004e+06,1.291713e+06,205498.444071,7.827094e+06,4.657223e+06,4.934178e+06,4.094341e+06,3.962330e+06,1.619288e+06,...,1.313619e+07,3.674675e+07,1.855573e+06,5.760053e+05,138007.572103,2.051916e+07,8.038182e+06,1.622416e+06,8.659168e+07,3.830890e+05
2003-3,249202.776314,1.601652e+06,1.298888e+06,205799.285261,7.871579e+06,4.678221e+06,4.971875e+06,4.119454e+06,3.989401e+06,1.628734e+06,...,1.315208e+07,3.674063e+07,1.867763e+06,5.805941e+05,138860.329589,2.057551e+07,8.074397e+06,1.626787e+06,8.666036e+07,3.851600e+05
2003-4,250224.776896,1.605299e+06,1.306063e+06,206100.126450,7.916064e+06,4.699218e+06,5.009571e+06,4.144566e+06,4.016472e+06,1.638180e+06,...,1.316797e+07,3.673451e+07,1.879953e+06,5.851830e+05,139713.087075,2.063185e+07,8.110613e+06,1.631158e+06,8.672905e+07,3.872309e+05
2003-5,251246.777478,1.608947e+06,1.313239e+06,206400.967640,7.960549e+06,4.720215e+06,5.047268e+06,4.169679e+06,4.043542e+06,1.647626e+06,...,1.318386e+07,3.672839e+07,1.892142e+06,5.897718e+05,140565.844561,2.068820e+07,8.146828e+06,1.635529e+06,8.679773e+07,3.893019e+05
2003-6,252268.778060,1.612595e+06,1.320414e+06,206701.808830,8.005034e+06,4.741212e+06,5.084964e+06,4.194792e+06,4.070613e+06,1.657072e+06,...,1.319975e+07,3.672227e+07,1.904332e+06,5.943606e+05,141418.602046,2.074454e+07,8.183043e+06,1.639900e+06,8.686642e+07,3.913728e+05
2003-7,253290.778642,1.616242e+06,1.327590e+06,207002.650019,8.049519e+06,4.762209e+06,5.122661e+06,4.219904e+06,4.097683e+06,1.666518e+06,...,1.321565e+07,3.671615e+07,1.916522e+06,5.989494e+05,142271.359532,2.080089e+07,8.219259e+06,1.644271e+06,8.693510e+07,3.934438e+05
2003-8,254312.779224,1.619890e+06,1.334765e+06,207303.491209,8.094004e+06,4.783207e+06,5.160357e+06,4.245017e+06,4.124754e+06,1.675964e+06,...,1.323154e+07,3.671003e+07,1.928711e+06,6.035383e+05,143124.117018,2.085723e+07,8.255474e+06,1.648642e+06,8.700379e+07,3.955147e+05
2003-9,255334.779806,1.623538e+06,1.341940e+06,207604.332399,8.138489e+06,4.804204e+06,5.198054e+06,4.270130e+06,4.151824e+06,1.685410e+06,...,1.324743e+07,3.670391e+07,1.940901e+06,6.081271e+05,143976.874504,2.091358e+07,8.291689e+06,1.653013e+06,8.707247e+07,3.975856e+05
2003-10,256356.780388,1.627186e+06,1.349116e+06,207905.173588,8.182974e+06,4.825201e+06,5.235750e+06,4.295242e+06,4.178895e+06,1.694856e+06,...,1.326332e+07,3.669778e+07,1.953091e+06,6.127159e+05,144829.631990,2.096992e+07,8.327904e+06,1.657384e+06,8.714116e+07,3.996566e+05


In [11]:
for column in df_mo_2003.columns:
    file_name = f"2003_mo_dep_poup_{column}.csv"
    column_df = df_mo_2003[[column]]
    column_df.to_csv(file_name, index=True)